In [16]:
import json
import os
input_path = '../dataset/collecte.json'
output_path = '../generated/normalized/collecte.json'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(input_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

for feature in data.get('features', []):
    props = feature.get('properties', {})
    if props.get('MUNICIPALITE') == 'Montreal':
        props['PROVINCE'] = 'Québec'

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
print(f"Fichier modifié enregistré sous {output_path}")

Fichier modifié enregistré sous ../generated/normalized/collecte.json


In [ ]:
import pandas as pd
# Extraction des données du Québec uniquement
df = pd.read_csv('../dataset/Matieres_residuelles_recuperees.csv', sep=';')
df_quebec = df[df['GÉO'].str.contains('Québec', case=False, na=False)].copy()
print(f"Nombre de lignes pour le Québec : {len(df_quebec)}")
df_quebec.to_csv('../generated/normalized/matieres_residuelles_quebec.csv', index=False)
print("✅ Fichier 'matieres_residuelles_quebec.csv' généré avec succès.")

Nombre de lignes pour le Québec : 162
✅ Fichier 'matieres_residuelles_quebec.csv' généré avec succès.


In [ ]:
# Normalisation des coordonnées GPS des centres de tri via OpenStreetMap Nominatim
import time
import requests
import json
import pandas as pd
input_path = '../dataset/liste-centres-tri-crd.csv'
output_path = '../generated/normalized/liste-centres-tri-crd.csv'
cache_file = '../cache/geocode_cache.csv'
df_sites = pd.read_csv(input_path)
df_sites['Code Postal '] = df_sites['Code Postal '].astype(str)
# On force les colonnes latitude/longitude à None si elles n'existent pas
if 'latitude' not in df_sites.columns:
    df_sites['latitude'] = None
if 'longitude' not in df_sites.columns:
    df_sites['longitude'] = None
# Charger le cache existant si disponible
os.makedirs(os.path.dirname(cache_file), exist_ok=True)
if os.path.exists(cache_file):
    with open(cache_file, 'r', encoding='utf-8') as f:
        geocode_cache = json.load(f)
else:
    geocode_cache = {}

def geocode_address(row):
    # Si déjà géocodé dans le DataFrame, ne rien faire
    if pd.notnull(row.get('latitude')) and pd.notnull(row.get('longitude')):
        return row['latitude'], row['longitude']
    address = f"{row['Adresse ']}, {row['Municipalité ']}, {row['Code Postal ']}, Québec, Canada"
    if address in geocode_cache:
        print(f"Cache hit for: {address}")
        return geocode_cache[address]
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'limit': 1,
    }
    response = requests.get(url, params=params, headers={'User-Agent': 'geo-app'})
    if response.status_code == 200 and response.json():
        data = response.json()[0]
        lat, lon = data['lat'], data['lon']
        geocode_cache[address] = (lat, lon)
        # Sauvegarder le cache à chaque nouvelle requête avec accents non échappés
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(geocode_cache, f, ensure_ascii=False, indent=2)
        return lat, lon
    return None, None

for i, row in df_sites.iterrows():
    lat, lon = geocode_address(row)
    df_sites.at[i, 'latitude'] = lat
    df_sites.at[i, 'longitude'] = lon
    print(f"{row['Adresse ']} → {lat}, {lon}")
    time.sleep(1.1)

df_sites.to_csv(output_path, index=False)
print("✅ Fichier 'liste-centres-tri-crd.csv' mis à jour avec succès.")

Cache hit for: 75, rue Savard, Matane, G4W 0H9, Québec, Canada
75, rue Savard → 48.8281270, -67.5722700
Cache hit for: 10, rue Charles-St-Pierre, Rivière-du-Loup, G5R 0N6, Québec, Canada
10, rue Charles-St-Pierre → 47.8398167, -69.5048167
Cache hit for: 10, rue Charles-St-Pierre, Rivière-du-Loup, G5R 0N6, Québec, Canada
10, rue Charles-St-Pierre → 47.8398167, -69.5048167
Cache hit for: 1125, rue des Pins, Alma, G8B 7V7, Québec, Canada
1125, rue des Pins → 48.5732353, -71.6583744
Cache hit for: 1125, rue des Pins, Alma, G8B 7V7, Québec, Canada
1125, rue des Pins → 48.5732353, -71.6583744
Cache hit for: 220, rue Rotterdam, Saint-Augustin-de-Desmaures, G3A 1T4, Québec, Canada
220, rue Rotterdam → 46.7585422, -71.4596075
Cache hit for: 220, rue Rotterdam, Saint-Augustin-de-Desmaures, G3A 1T4, Québec, Canada
220, rue Rotterdam → 46.7585422, -71.4596075
Cache hit for: 315, rue Jackson, Québec, G1N 4C4, Québec, Canada
315, rue Jackson → 46.8028828, -71.2634463
Cache hit for: 315, rue Jackson,